<a href="https://colab.research.google.com/github/AyaTarek29/BigMart-Data-Analysis-and-Prediction/blob/main/IntTask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Problem Statment: BigMart is a retail chain with 10 outlets in different cities. They have collected sales data for 1559 products from 2013. The data scientists at BigMart want to build a predictive model to predict the sales of each product at a particular outlet. This model will help BigMart understand the properties of products and outlets that play a key role in increasing sales.
#Item weight
#Items fat content
#Item_Visibility
#Item MRP:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")
upload=files.upload()
upload=files.upload()

In [ ]:
df_train = pd.read_csv("Train.csv")
df_test = pd.read_csv("Test.csv")

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:

print('Train Data',df_train.shape)
print('Test Data',df_test.shape)

In [ ]:

df_train.info()

In [ ]:
df_train.boxplot()

In [ ]:
print('Number of trainings examples:', len(df_train),'\n')
df_train.describe().T.style.background_gradient(cmap='Blues')

In [ ]:
numerical = df_train.select_dtypes(include = ['int64', 'Int64','float64']).dtypes.index
numerical

In [ ]:
numerical = df_train.select_dtypes(include = ['int64', 'Int64','float64']).dtypes.index
numerical

In [ ]:
def UVA_Categorical(data, cat):
  plt.figure(figsize = (10,6))
  sns.countplot(x=cat, data = data)
  plt.xlabel(cat,fontsize = 14, fontweight = 'bold')
  plt.ylabel('Count',fontsize = 14, fontweight = 'bold')
  plt.title('Value counts: \n{}'.format(df_train[cat].value_counts(normalize = True)))
  if len(data[cat].value_counts()) > 7:
    X = plt.gca().xaxis
    for item in X.get_ticklabels():
      item.set_rotation(90)
  plt.show()

In [ ]:

UVA_Categorical(df_train,'Item_Fat_Content')

In [ ]:
total_low_fat = 0.597090 + 0.037076 + 0.013141
total_low_fat

In [ ]:
UVA_Categorical(df_train, 'Item_Type')

In [ ]:
UVA_Categorical(df_train, 'Outlet_Identifier')

In [ ]:
UVA_Categorical(df_train,'Outlet_Size')

In [ ]:
UVA_Categorical(df_train, 'Outlet_Location_Type')

In [ ]:
UVA_Categorical(df_train, 'Outlet_Type')

In [ ]:
df_train['source'] = 'Train'
df_test['source'] = 'Test'
df=pd.concat([df_train,df_test], ignore_index=True)

In [ ]:
df.info()

In [ ]:
plt.figure(figsize = (10,6))
sns.heatmap(df.isnull(), yticklabels=False,cbar = False,cmap ='viridis')

In [ ]:
def missing_percent():
  miss_item_weight = (df['Item_Weight'].isnull().sum()/len(df))*100
  miss_Outlet_Size = (df['Outlet_Size'].isnull().sum()/len(df))*100

  print('% of missing values in Item_Weight: ' + str(miss_item_weight))
  print('% of missing values in Outlet_Size: ' +str(miss_Outlet_Size))

missing_percent()

In [ ]:
df['Item_Weight'].fillna(df['Item_Weight'].mean(),inplace=True)

In [ ]:
df['Outlet_Size'].fillna('Medium', inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.Item_Visibility.value_counts

In [ ]:
df['Item_Visibility'].replace(0.0,value=np.nan,inplace=True)

In [ ]:
df['Item_Visibility']=df['Item_Visibility'].fillna(df.groupby('Item_Identifier')['Item_Visibility'].transform('mean'))

In [ ]:
plt.figure(figsize=(7,5))
sns.countplot(x='Item_Fat_Content',data=df)

In [ ]:
df['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'},inplace=True)

In [ ]:
plt.figure(figsize=(7,5))
sns.countplot(x='Item_Fat_Content',data=df)

In [ ]:
test_pred = df.loc[df['source'] == 'Test']
test_pred.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
test_pred.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
categ = ['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
le = LabelEncoder()
df[categ] = df[categ].apply(le.fit_transform)

In [ ]:
df.drop(['Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
train = df.loc[df['source'] == 'Train']
test = df.loc[df['source'] == 'Test']

In [ ]:
train.drop(['source'],axis=1,inplace=True)
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
x = train.drop(columns="Item_Outlet_Sales")
y = train["Item_Outlet_Sales"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state = 0)

In [ ]:
model_comparison = {}

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
y_pred

In [ ]:
model_comparison['Linear Regression'] = [lr.score(X_train,y_train)*100,r2_score(y_test,y_pred)*100]

print("Linear Regression\n\nAccuracy: {}%".format(round(lr.score(X_train,y_train)*100)))
print("r2 score: {}%".format(round(r2_score(y_test,y_pred)*100)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=400,max_depth=6,min_samples_leaf=100,n_jobs=4)
rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

In [ ]:
y_pred

In [ ]:
model_comparison['Random Forest'] = [rf.score(X_train,y_train)*100,r2_score(y_test,y_pred)*100]

print("Random Forest\n\nAccuracy: {}%".format(round(rf.score(X_train,y_train)*100)))
print("r2 score: {}%".format(round(r2_score(y_test,y_pred)*100)))

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators = 100, learning_rate=0.05)
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

In [ ]:
y_pred

In [ ]:
model_comparison['XGBoost Regressor'] = [xgb.score(X_train,y_train)*100,r2_score(y_test,y_pred)*100]

print("XGBoost Regressor\n\nAccuracy: {}%".format(round(xgb.score(X_train,y_train)*100)))
print("r2 score: {}%".format(round(r2_score(y_test,y_pred)*100)))

In [ ]:
model_comparison_df = pd.DataFrame.from_dict(model_comparison).T
model_comparison_df.columns = ['Accuracy', "r2_score"]
model_comparison_df = model_comparison_df.sort_values('Accuracy', ascending=True)
model_comparison_df.style.background_gradient(cmap='Blues')

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='r2_score', y=model_comparison_df.index, x=model_comparison_df['r2_score'], orientation='h', marker_color='#7baaf7'),
    go.Bar(name='Accuracy', y=model_comparison_df.index, x=model_comparison_df['Accuracy'], orientation='h', marker_color='#4285f4')
])
fig.update_layout(barmode='group')
fig.show()

In [ ]:
test.head()

In [ ]:
pred = xgb.predict(test)

In [ ]:
pred

In [ ]:
test_pred.head()

In [ ]:
test_pred["Predicted_Item_Outlet_Sale"] = pred

In [ ]:
test_pred

In [ ]:
test_pred.to_csv("submission.csv",index=False)

In [ ]:
ls=[]
for i in test.columns:
    s = float(input(f"Enter the {i}:"))
    ls.append(s)

In [ ]:
xgb.predict(np.array(ls).reshape(1,-1))

In [ ]:
ls1=[]
for i in test.columns:
    s = float(input(f"Enter the {i}:"))
    ls1.append(s)

In [ ]:
xgb.predict(np.array(ls1).reshape(1,-1))

In [ ]:
import joblib
import pickle

joblib.dump(xgb,"BigMart_model.sav")

In [ ]:
pickle.dump(xgb,open('BigMart_model.pkl','wb'))